In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
%cd drive/MyDrive/Colab Notebooks/Thesis Project/

/content/drive/MyDrive/Colab Notebooks/Thesis Project


In [ ]:
!ls

'0. Depression Analysis Dataset.xlsx'
'1. Depression Analysis Dataset Cleaned v1.xlsx'
'1. Depression Analysis Dataset Cleaned v2.xlsx'
'1. Depression Analysis Dataset Cleaned v3_textcleaned.xlsx'
'1. Depression Analysis Dataset Cleaned v3_textstemmed.xlsx'
'1. Depression Analysis v1.ipynb'
'2.1 Depression Analysis v2 (Previous code).ipynb'
'2.2 Depression Analysis v2.ipynb'
'2.3 Depression Analysis v2 modified.ipynb'
'3.1 Depression_Analysis_v3.ipynb'
'3.2 Depression_Analysis_v3 (Word2Vec).ipynb'
'3.3 Depression_Analysis_v3 (FastText).ipynb'
'3.4 Depression_Analysis_LSTM_FastText.ipynb'
'3.4 Depression_Analysis_v3 (Glove).ipynb'
'3.5 Depression_Analysis_BiLSTM_FastText.ipynb'
'4.1 Depression_Analysis_v4 (New Tokenizer).ipynb'
'4.2 Depression_Analysis_v4_Lemmatize.ipynb'
'4.3 Depression_Analysis_v4_Stem.ipynb'
'4.4 Depression_Analysis (Hyperparameter Tuning).ipynb'
'5. BanglaBert.ipynb'
 content
'Depression_Analysis (Bayesian Optimization).ipynb'
 preprocessed_dataset.csv
 state_dict.p

In [ ]:
pip install transformers


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 30.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 23.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 86.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 34.0 MB/s eta 0:00:00


In [ ]:
#import libraries
import numpy as np
import pandas as pd
import random
import re, string
import seaborn as sns
import matplotlib.pyplot as plt

#PyTorch
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from sklearn.model_selection import train_test_split


from transformers import AutoTokenizer, AutoModelForSequenceClassification, AdamW


In [ ]:
# Load the dataset
df = pd.read_excel('1. Depression Analysis Dataset Cleaned v2.xlsx')
df.head()

,text,label
0,লক্ষ্য জীবনের মূল্য সারাদিন কাটাই দুঃখ বেদনায়...,1
1,ক্রমাগত নিজেকে পুনরাবৃত্তি লড়াই সুযোগ সত্যই প...,1
2,বয়সী বয়স বিষণ্নতায় গভীরভাবে বেড়ে উঠেছি কঠি...,1
3,তুরস্কে বসবাসকারী বয়স সম্ভবত পোস্ট করছি সিদ্ধ...,1
4,কিছুক্ষণ ঘটেছিল বিরক্ত সাহায্য কথা নোংরা সত্যি...,1


In [ ]:
df.to_csv('preprocessed_dataset.csv', index=False)


In [ ]:
# Load preprocessed dataset
df = pd.read_csv('preprocessed_dataset.csv')

# Split dataset into training and evaluation sets
train_df, eval_df = train_test_split(df, test_size=0.2, random_state=42)

# Load the BanglaBERT tokenizer
tokenizer = AutoTokenizer.from_pretrained('sagorsarker/bangla-bert-base')

In [ ]:
# Set the maximum sequence length
max_seq_length = 512

# Tokenize and encode text data with max length
train_encodings = tokenizer(list(train_df['text']), truncation=True, padding='max_length', max_length=max_seq_length)
eval_encodings = tokenizer(list(eval_df['text']), truncation=True, padding='max_length', max_length=max_seq_length)

# Convert encodings to tensors
train_input_ids = torch.tensor(train_encodings['input_ids'])
train_attention_mask = torch.tensor(train_encodings['attention_mask'])
eval_input_ids = torch.tensor(eval_encodings['input_ids'])
eval_attention_mask = torch.tensor(eval_encodings['attention_mask'])

# Convert labels to tensors
train_labels = torch.tensor(list(train_df['label']))
eval_labels = torch.tensor(list(eval_df['label']))

# Create PyTorch datasets
train_dataset = torch.utils.data.TensorDataset(train_input_ids, train_attention_mask, train_labels)
eval_dataset = torch.utils.data.TensorDataset(eval_input_ids, eval_attention_mask, eval_labels)


In [ ]:
# Load the BanglaBERT model for sequence classification
model = AutoModelForSequenceClassification.from_pretrained('sagorsarker/bangla-bert-base', num_labels=2)

# Set up the optimizer
optimizer = AdamW(model.parameters(), lr=1e-3)

# Set up the data loaders
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=8, shuffle=True)
eval_loader = torch.utils.data.DataLoader(eval_dataset, batch_size=8, shuffle=False)

# Fine-tuning BanglaBERT
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

Some weights of the model checkpoint at sagorsarker/bangla-bert-base were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at sagorsarker/bangla-ber

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(102025, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12

In [ ]:
for epoch in range(3):  # Set the number of training epochs
    model.train()
    for batch in train_loader:
        input_ids, attention_mask, labels = batch
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

    model.eval()
    eval_accuracy = 0
    eval_loss = 0
    with torch.no_grad():
        for batch in eval_loader:
            input_ids, attention_mask, labels = batch
            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)
            labels = labels.to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            logits = outputs.logits

            eval_loss += outputs.loss.item()
            eval_accuracy += (logits.argmax(dim=1) == labels).float().mean().item()

    eval_loss /= len(eval_loader)
    eval_accuracy /= len(eval_loader)
    print(f"Epoch {epoch+1}: Evaluation Loss: {eval_loss}, Evaluation Accuracy: {eval_accuracy}")


Epoch 1: Evaluation Loss: 0.9217860773371694, Evaluation Accuracy: 0.5101018135823757
Epoch 2: Evaluation Loss: 0.6934935763843341, Evaluation Accuracy: 0.5101018135823757
Epoch 3: Evaluation Loss: 0.6933695972629539, Evaluation Accuracy: 0.5101018135823757
